Generación de dataset

In [1]:
import numpy as np
import pandas as pd

rng = np.random.default_rng(42)

# -----------------------------
# 1) CLIENTES (similar al archivo)
# -----------------------------
n_clientes = 500

ids = np.arange(1, n_clientes + 1)

nombres_base = np.array([
    "Ana", "Sofía", "Camila", "Valentina", "Isidora", "Martina",
    "Mateo", "Benjamín", "Tomás", "Vicente", "Lucas", "Joaquín",
    "Daniela", "Fernanda", "Ignacio", "Gabriel", "Antonia", "Catalina"
])

apellidos_base = np.array([
    "González", "Muñoz", "Rojas", "Díaz", "Pérez", "Soto",
    "Contreras", "Silva", "Martínez", "Sepúlveda", "Torres", "Flores"
])

nombres = rng.choice(nombres_base, size=n_clientes) + " " + rng.choice(apellidos_base, size=n_clientes)

ciudades = np.array(["Santiago", "Valparaíso", "Concepción", "La Serena", "Antofagasta", "Temuco", "Rancagua"])
ciudad = rng.choice(ciudades, size=n_clientes, p=[0.45, 0.12, 0.14, 0.08, 0.08, 0.08, 0.05])

# Edad: distribución razonable para ecommerce (18–70)
edad = np.clip(rng.normal(loc=35, scale=12, size=n_clientes), 18, 70).round(0).astype(int)

# Total_Compras: más realista con Poisson (muchos con pocas compras, pocos con muchas)
total_compras = rng.poisson(lam=4.5, size=n_clientes)
total_compras = np.clip(total_compras, 0, 35)

# Ticket promedio (CLP) por cliente: lognormal para sesgo a la derecha (algunos gastan mucho)
ticket_promedio = rng.lognormal(mean=np.log(25000), sigma=0.55, size=n_clientes)

# Monto_Total = Total_Compras * Ticket_promedio + ruido
monto_total = (total_compras * ticket_promedio) + rng.normal(0, 5000, size=n_clientes)
monto_total = np.clip(monto_total, 0, None).round(0).astype(int)

clientes = pd.DataFrame({
    "ID": ids,
    "Nombre": nombres,
    "Edad": edad,
    "Ciudad": ciudad,
    "Total_Compras": total_compras,
    "Monto_Total": monto_total
})

# -----------------------------
# 2) TRANSACCIONES (opcional, pero recomendado)
# -----------------------------
# Genera una tabla de transacciones coherente con Total_Compras y Monto_Total.
# Ojo: si Total_Compras=0, ese cliente no tendrá transacciones.

n_transacciones = int(total_compras.sum())

# Repetimos el ID del cliente según su cantidad de compras
tx_id_cliente = np.repeat(ids, total_compras)

# Fechas aleatorias en los últimos 365 días
dias_atras = rng.integers(0, 365, size=n_transacciones)
tx_fecha = (pd.Timestamp.today().normalize() - pd.to_timedelta(dias_atras, unit="D")).astype("datetime64[ns]")

# Montos por transacción: repartimos el monto total del cliente en sus compras (con variación)
tx_monto = np.empty(n_transacciones, dtype=float)

start = 0
for i, c in enumerate(total_compras):
    if c == 0:
        continue
    end = start + c

    # pesos aleatorios para repartir (Dirichlet => suman 1)
    pesos = rng.dirichlet(alpha=np.ones(c) * 1.3)
    montos = pesos * monto_total[i]

    # ruido leve y mínimo razonable
    montos = np.clip(montos + rng.normal(0, 1500, size=c), 1500, None)
    tx_monto[start:end] = montos
    start = end

tx_monto = tx_monto.round(0).astype(int)

# Categorías ficticias
categorias = np.array(["Electrónica", "Hogar", "Moda", "Deportes", "Juguetes", "Belleza", "Alimentos"])
tx_categoria = rng.choice(categorias, size=n_transacciones, p=[0.18, 0.16, 0.20, 0.12, 0.10, 0.14, 0.10])

transacciones = pd.DataFrame({
    "ID_Transaccion": np.arange(1, n_transacciones + 1),
    "ID_Cliente": tx_id_cliente,
    "Fecha": tx_fecha,
    "Categoria": tx_categoria,
    "Monto": tx_monto
})

# -----------------------------
# Guardar a CSV (si lo necesitas)
# -----------------------------
#clientes.to_csv("clientes_ficticios.csv", index=False, encoding="utf-8")
#transacciones.to_csv("transacciones_ficticias.csv", index=False, encoding="utf-8")



ENSUCIAR DATOS

In [2]:
# A) NULOS en clientes
idx_null_edad = rng.choice(clientes.index, size=int(0.03 * n_clientes), replace=False)
clientes.loc[idx_null_edad, "Edad"] = np.nan


idx_null_ciudad = rng.choice(clientes.index, size=int(0.02 * n_clientes), replace=False)
clientes.loc[idx_null_ciudad, "Ciudad"] = None


# B) Formato inconsistente en Ciudad (mayúsculas / minúsculas / espacios)
idx_messy_city = rng.choice(clientes.index, size=int(0.05 * n_clientes), replace=False)
clientes.loc[idx_messy_city, "Ciudad"] = (
clientes.loc[idx_messy_city, "Ciudad"]
.astype(str)
.str.upper()
.str.strip()
)


idx_spaces_city = rng.choice(clientes.index, size=int(0.03 * n_clientes), replace=False)
clientes.loc[idx_spaces_city, "Ciudad"] = clientes.loc[idx_spaces_city, "Ciudad"].astype(str) + " "


# C) Outliers en Edad (unos pocos)
idx_out_edad = rng.choice(clientes.index, size=4, replace=False)
clientes.loc[idx_out_edad, "Edad"] = [0, 120, 150, 95]


# D) NULOS en transacciones (Categoria)
idx_null_cat = rng.choice(transacciones.index, size=int(0.02 * len(transacciones)), replace=False)
transacciones.loc[idx_null_cat, "Categoria"] = None


# E) Formato inconsistente en Fecha: convertir algunas a string con formato distinto
idx_str_fecha = rng.choice(transacciones.index, size=int(0.03 * len(transacciones)), replace=False)
transacciones.loc[idx_str_fecha, "Fecha"] = transacciones.loc[idx_str_fecha, "Fecha"].dt.strftime("%d-%m-%Y")


# F) Outliers en Monto: inflar algunos montos
idx_out_monto = rng.choice(transacciones.index, size=max(5, int(0.005 * len(transacciones))), replace=False)
transacciones.loc[idx_out_monto, "Monto"] = transacciones.loc[idx_out_monto, "Monto"] * 30


# G) Duplicados en transacciones: duplicar algunas filas
dup_rows = transacciones.sample(frac=0.02, random_state=42)
transacciones = pd.concat([transacciones, dup_rows], ignore_index=True)


# (opcional) mezclar filas
transacciones = transacciones.sample(frac=1, random_state=42).reset_index(drop=True)

In [3]:
transacciones.shape

(2265, 5)

In [4]:
clientes.shape

(500, 6)